# Modèle de prédiction popularité jeux vidéo 🎮
### 🎯 Contexte & question business
Dataset : Les données ont été scrappées via l'API Steam et regroupe des info jeux vidéo comme prix, genres, plateformes aussi des indicateurs d’avis et d’engagement des joueurs.

Question :  Dans quelle mesure peut-on prédire la popularité d’un jeu vidéo sur Steam, mesurée par les avis des joueurs, à partir de ses caractéristiques (prix, genres, plateformes, contenu) ?

KPI's :
- Performance du modèle (R², RSME)
- Leviers (coeff)
- Segments (genres, prix, plateforme)

### 🧩 Variables principales
id: Identifiant unique
name: Titre du jeu
released: Date de sortie
updated: Dernière mise à jour
rating: Note moyenne des joueurs
suggestions_count: Nombre de recommandations
platforms: Plateformes disponibles (PC, PlayStation, Xbox, Switch, Mobile…)
genres: Genre(s) du jeu (RPG, Action, Adventure, Indie…)
developers: Studio(s) de développement
publishers: Éditeur(s)
description: Résumé du jeu



### ⚙️ Imports et Setup

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

### ⚡Ouverture et Exploration du Dataset

In [4]:
df = pd.read_csv('games_march2025_cleaned.csv', engine='python')
df.head()

ParserError: unexpected end of data

In [ ]:
df.shape

(89618, 47)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89618 entries, 0 to 89617
Data columns (total 47 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   appid                     89618 non-null  int64  
 1   name                      89618 non-null  object 
 2   release_date              89618 non-null  object 
 3   required_age              89618 non-null  int64  
 4   price                     89618 non-null  float64
 5   dlc_count                 89618 non-null  int64  
 6   detailed_description      89421 non-null  object 
 7   about_the_game            89398 non-null  object 
 8   short_description         89498 non-null  object 
 9   reviews                   10401 non-null  object 
 10  header_image              89618 non-null  object 
 11  website                   41114 non-null  object 
 12  support_url               44110 non-null  object 
 13  support_email             78798 non-null  object 
 14  window

In [ ]:
df.describe()

,appid,required_age,price,dlc_count,metacritic_score,achievements,recommendations,user_score,score_rank,positive,...,average_playtime_forever,average_playtime_2weeks,median_playtime_forever,median_playtime_2weeks,discount,peak_ccu,pct_pos_total,num_reviews_total,pct_pos_recent,num_reviews_recent
count,8.961800e+04,89618.000000,89618.000000,89618.000000,89618.000000,89618.000000,8.961800e+04,89618.000000,39.000000,8.961800e+04,...,8.961800e+04,89618.000000,8.961800e+04,89618.000000,89618.000000,8.961800e+04,89618.000000,8.961800e+04,89618.000000,89618.000000
mean,1.656904e+06,0.183624,7.309623,0.595583,2.903245,20.552333,1.009401e+03,0.032817,99.128205,1.269896e+03,...,1.149119e+02,5.032694,1.147559e+02,5.301814,4.558627,9.833994e+01,45.352418,1.315490e+03,5.231639,15.584581
std,9.168390e+05,1.725594,13.331073,15.351920,14.445358,163.562418,2.204815e+04,1.615149,0.695076,3.181441e+04,...,6.814748e+03,181.100334,8.806711e+03,189.591095,16.538036,5.717544e+03,40.685695,3.542370e+04,22.335358,444.277307
min,2.000000e+01,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,98.000000,0.000000e+00,...,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,-1.000000,-1.000000e+00,-1.000000,-1.000000
25%,8.550525e+05,0.000000,0.990000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,99.000000,1.000000e+00,...,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,-1.000000,-1.000000e+00,-1.000000,-1.000000
50%,1.524730e+06,0.000000,4.990000,0.000000,0.000000,5.000000,0.000000e+00,0.000000,99.000000,1.000000e+01,...,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,60.000000,1.500000e+01,-1.000000,-1.000000
75%,2.430852e+06,0.000000,9.990000,0.000000,0.000000,20.000000,0.000000e+00,0.000000,100.000000,5.800000e+01,...,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,84.000000,8.100000e+01,-1.000000,-1.000000
max,3.542350e+06,21.000000,999.980000,3427.000000,97.000000,9821.000000,4.401572e+06,100.000000,100.000000,7.480813e+06,...,1.462997e+06,18568.000000,1.462997e+06,18568.000000,100.000000,1.212356e+06,100.000000,8.632939e+06,100.000000,96473.000000


### 🧹Nettoyage du Dataset

In [ ]:
df = df[["release_date","required_age", "price", "discount", "dlc_count","achievements", "metacritic_score","windows", "mac", "linux","positive", "negative","pct_pos_total", "pct_pos_recent",
    "num_reviews_total", "num_reviews_recent","average_playtime_forever", "average_playtime_2weeks","median_playtime_forever", "median_playtime_2weeks","peak_ccu","genres","user_score"]]
df.head(2)


,release_date,required_age,price,discount,dlc_count,achievements,metacritic_score,windows,mac,linux,...,pct_pos_recent,num_reviews_total,num_reviews_recent,average_playtime_forever,average_playtime_2weeks,median_playtime_forever,median_playtime_2weeks,peak_ccu,genres,user_score
0,2012-08-21,0,0.0,0,1,1,0,True,False,True,...,82,8632939,96473,33189,879,5174,350,1212356,"['Action', 'Free To Play']",0
1,2017-12-21,0,0.0,0,0,37,0,True,False,False,...,68,2513842,16720,0,0,0,0,616738,"['Action', 'Adventure', 'Massively Multiplayer...",0


In [ ]:
df.isna().sum()

,0
release_date,0
required_age,0
price,0
discount,0
dlc_count,0
achievements,0
metacritic_score,0
windows,0
mac,0
linux,0


In [ ]:
df["release_date"] = pd.to_datetime(df["release_date"], errors="coerce")
df["release_year"] = df["release_date"].dt.year
df = df.drop("release_date", axis=1)
df[["release_year"]].head(2)

,release_year
0,2012
1,2017


In [ ]:
df.columns

Index(['required_age', 'price', 'discount', 'dlc_count', 'achievements',
       'metacritic_score', 'windows', 'mac', 'linux', 'positive', 'negative',
       'pct_pos_total', 'pct_pos_recent', 'num_reviews_total',
       'num_reviews_recent', 'average_playtime_forever',
       'average_playtime_2weeks', 'median_playtime_forever',
       'median_playtime_2weeks', 'peak_ccu', 'genres', 'user_score',
       'release_year'],
      dtype='object')

In [ ]:
df["genres"] = df["genres"].fillna("Unknown")
df = df.fillna(0)

In [ ]:
df["genres_list"] = df["genres"].str.split(",")

df["main_genre"] = df["genres_list"].apply(lambda x: x[0].strip() if len(x) > 0 else "Unknown")
df["sub_genre"] = df["genres_list"].apply(lambda x: x[1].strip() if len(x) > 1 else "None")
df.head()

,required_age,price,discount,dlc_count,achievements,metacritic_score,windows,mac,linux,positive,...,average_playtime_2weeks,median_playtime_forever,median_playtime_2weeks,peak_ccu,genres,user_score,release_year,genres_list,main_genre,sub_genre
0,0,0.00,0,1,1,0,True,False,True,7480813,...,879,5174,350,1212356,"['Action', 'Free To Play']",0,2012,"[['Action', 'Free To Play']]",['Action','Free To Play']
1,0,0.00,0,0,37,0,True,False,False,1487960,...,0,0,0,616738,"['Action', 'Adventure', 'Massively Multiplayer...",0,2017,"[['Action', 'Adventure', 'Massively Multipla...",['Action','Adventure'
2,0,0.00,0,2,0,90,True,True,True,1998462,...,1536,898,892,555977,"['Action', 'Strategy', 'Free To Play']",0,2013,"[['Action', 'Strategy', 'Free To Play']]",['Action','Strategy'
3,17,0.00,0,0,77,96,True,False,False,1719950,...,771,7101,74,117698,"['Action', 'Adventure']",0,2015,"[['Action', 'Adventure']]",['Action','Adventure']
4,17,3.99,80,9,0,0,True,False,False,1152763,...,682,2434,306,89916,['Action'],0,2015,[['Action']],['Action'],None


In [ ]:
#Verfier les valeur unique pour mieux les regrouper par plateforme
print(df['genres'].unique())


["['Action', 'Free To Play']"
 "['Action', 'Adventure', 'Massively Multiplayer', 'Free To Play']"
 "['Action', 'Strategy', 'Free To Play']" ...
 "['Action', 'Indie', 'Simulation', 'Education', 'Game Development']"
 "['Design & Illustration', 'Education', 'Game Development']"
 "['Violent', 'Action', 'Early Access']"]


In [ ]:
all_genres = df["genres"].dropna().str.split(",").sum() # concat toutes les listes et séparer les genres car certains en ont plusieurs
counts = Counter([g.strip() for g in all_genres])
genre=counts.most_common()
print(genre)
print(all_genres)


KeyboardInterrupt: 

In [ ]:
df = df.drop(columns=["genres", "genres_list"])
df.head()

### 📲 Analyses EDA

#### Interprétations :

### ⚡ Modèle

#### Interprétations :

In [ ]:
# ============================================================
# PROJET ML - STEAM (Kaggle) - REGRESSION LINEAIRE (1 SEUL MODELE)
# Objectif : prédire la note utilisateur (user_score) d'un jeu Steam
# à partir de variables explicatives (avis, engagement, prix, genres, etc.)
# ============================================================

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score


# ------------------------------------------------------------
# 1) CHARGEMENT DES DONNEES
# ------------------------------------------------------------
df = pd.read_csv("steam_games.csv")

# Petit aperçu
print("Shape (lignes, colonnes) :", df.shape)
print("\nAperçu :")
print(df.head())


# ------------------------------------------------------------
# 2) SELECTION DES COLONNES UTILES
# (On évite les colonnes type url/images/textes pour rester simple)
# ------------------------------------------------------------
cols = [
    "release_date",
    "required_age", "price", "discount", "dlc_count",
    "achievements", "metacritic_score",
    "windows", "mac", "linux",
    "positive", "negative",
    "pct_pos_total", "pct_pos_recent",
    "num_reviews_total", "num_reviews_recent",
    "average_playtime_forever", "average_playtime_2weeks",
    "median_playtime_forever", "median_playtime_2weeks",
    "peak_ccu",
    "genres",
    "user_score"
]

df = df[cols].copy()

# ------------------------------------------------------------
# 3) NETTOYAGE SIMPLE
# ------------------------------------------------------------

# 3.1) On supprime les lignes où la cible (user_score) n'existe pas
df = df.dropna(subset=["user_score"])

# 3.2) Transformation de la date en année (plus simple à utiliser dans un modèle)
df["release_date"] = pd.to_datetime(df["release_date"], errors="coerce")
df["release_year"] = df["release_date"].dt.year

# On supprime la date brute (on garde seulement release_year)
df = df.drop(columns=["release_date"])

# 3.3) Vérifier les valeurs manquantes
print("\nValeurs manquantes par colonne (top 10) :")
print(df.isna().sum().sort_values(ascending=False).head(10))

# Ici, pour rester niveau lycée :
# - on remplit genres manquant par "Unknown"
# - et le reste par 0
df["genres"] = df["genres"].fillna("Unknown")
df = df.fillna(0)


# ------------------------------------------------------------
# 4) CREATION DES GENRES : MAIN GENRE + SUB GENRE (1 seul chacun)
# ------------------------------------------------------------
# Exemple : "Action,Adventure,RPG"
# main_genre = "Action"
# sub_genre = "Adventure"

df["genres_list"] = df["genres"].astype(str).str.split(",")

df["main_genre"] = df["genres_list"].apply(lambda x: x[0].strip() if len(x) > 0 else "Unknown")
df["sub_genre"] = df["genres_list"].apply(lambda x: x[1].strip() if len(x) > 1 else "None")

# On supprime les colonnes texte initiales
df = df.drop(columns=["genres", "genres_list"])


# ------------------------------------------------------------
# 5) EDA (EXPLORATION VISUELLE)
# ------------------------------------------------------------

# 5.1) Distribution de la variable cible
sns.histplot(df["user_score"], bins=20)
plt.title("Distribution de user_score (note utilisateur)")
plt.show()

# 5.2) Matrice de corrélation sur les variables numériques
plt.figure(figsize=(12, 8))
sns.heatmap(df.select_dtypes(include=[np.number]).corr(), cmap="coolwarm")
plt.title("Corrélation (variables numériques)")
plt.show()


# ------------------------------------------------------------
# 6) ENCODAGE DES VARIABLES CATEGORIELLES (MAIN / SUB GENRE)
# ------------------------------------------------------------
# On transforme main_genre et sub_genre en colonnes 0/1 avec get_dummies
df_encoded = pd.get_dummies(df, columns=["main_genre", "sub_genre"], drop_first=True)

print("\nAprès encodage, nombre de colonnes :", df_encoded.shape[1])


# ------------------------------------------------------------
# 7) PREPARATION POUR LE MODELE (X / y)
# ------------------------------------------------------------
X = df_encoded.drop("user_score", axis=1)  # variables explicatives
y = df_encoded["user_score"]               # variable cible


# ------------------------------------------------------------
# 8) SPLIT TRAIN / TEST
# ------------------------------------------------------------
# 80% entraînement, 20% test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print("\nTaille train :", X_train.shape, " | Taille test :", X_test.shape)


# ------------------------------------------------------------
# 9) NORMALISATION
# ------------------------------------------------------------
# StandardScaler : utile pour la régression linéaire (mettre à la même échelle)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


# ------------------------------------------------------------
# 10) MODELE : REGRESSION LINEAIRE
# ------------------------------------------------------------
model = LinearRegression()
model.fit(X_train_scaled, y_train)

# Prédictions sur le test
y_pred = model.predict(X_test_scaled)


# ------------------------------------------------------------
# 11) EVALUATION DU MODELE (KPI)
# ------------------------------------------------------------
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("\n=== KPI (Test set) ===")
print("RMSE :", rmse)
print("MAE  :", mae)
print("R2   :", r2)


# ------------------------------------------------------------
# 12) INTERPRETATION (COEFFICIENTS)
# ------------------------------------------------------------
# Les coefficients indiquent l'influence de chaque variable sur la note (approx.)
coef = pd.Series(model.coef_, index=X.columns).sort_values()

print("\nTop 10 variables qui augmentent le plus la note (coeff positifs) :")
print(coef.tail(10))

print("\nTop 10 variables qui baissent le plus la note (coeff négatifs) :")
print(coef.head(10))


# ------------------------------------------------------------
# 13) VISU : REEL vs PREDIT (simple)
# ------------------------------------------------------------
plt.scatter(y_test, y_pred)
plt.xlabel("user_score réel")
plt.ylabel("user_score prédit")
plt.title("Réel vs Prédit (Régression Linéaire)")
plt.show()


# ------------------------------------------------------------
# 14) SIMULATION (ESTIMATION) - OPTIONNELLE MAIS CREATIVE
# ------------------------------------------------------------
# Ici on crée un "jeu fictif" en donnant quelques valeurs.
# Attention : ce n'est PAS une vraie note réelle, juste une estimation du modèle.

# Exemple d'inputs
user_main_genre = "Action"
user_sub_genre = "Adventure"
user_release_year = 2026  # extrapolation possible

new_game = pd.DataFrame(columns=X.columns)
new_game.loc[0] = 0

# Remplir quelques valeurs numériques (modifiable)
new_values = {
    "required_age": 16,
    "price": 29.99,
    "discount": 0,
    "dlc_count": 1,
    "achievements": 25,
    "metacritic_score": 78,
    "windows": 1,
    "mac": 0,
    "linux": 0,
    "positive": 1200,
    "negative": 300,
    "pct_pos_total": 80,
    "pct_pos_recent": 82,
    "num_reviews_total": 1500,
    "num_reviews_recent": 400,
    "average_playtime_forever": 420,
    "average_playtime_2weeks": 180,
    "median_playtime_forever": 300,
    "median_playtime_2weeks": 150,
    "peak_ccu": 3500,
    "release_year": user_release_year
}

for k, v in new_values.items():
    if k in new_game.columns:
        new_game.loc[0, k] = v

# Remplir les colonnes encodées pour main/sub genre si elles existent
main_col = f"main_genre_{user_main_genre}"
sub_col = f"sub_genre_{user_sub_genre}"

if main_col in new_game.columns:
    new_game.loc[0, main_col] = 1

if sub_col in new_game.columns:
    new_game.loc[0, sub_col] = 1

# Appliquer la même normalisation
new_game_scaled = scaler.transform(new_game)

# Prédire une note estimée
predicted_user_score = model.predict(new_game_scaled)[0]

print("\n=== Simulation (estimation) ===")
print("Main genre :", user_main_genre, "| Sub genre :", user_sub_genre, "| Année :", user_release_year)
print("Estimation de user_score :", round(predicted_user_score, 2))
